### Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [224]:
np.set_printoptions(threshold=np.nan)

import warnings
import traceback

warnings.filterwarnings('ignore')
import argparse
import re
import sys, os

sys.path.append(os.getcwd())

import os
import requests
from requests.exceptions import ConnectionError

import bs4
from bs4 import BeautifulSoup
from fastnumbers import isfloat
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import more_itertools
from random import shuffle





### Libs

In [3]:
def get_ci(p,t,r):
    return np.abs(np.fv(r/100,t,0,p))

def get_cumulative_amounts(p,t,r):
    psum = p
    for i in range(1,t):
        psum = psum + get_ci(p,i,r)
    return psum

def get_year_when_cumulative_profit_over_pe(pe,cpg):
    if np.isnan(pe):
        return np.inf
    for i in range(1,int(np.ceil(pe))):
        if get_cumulative_amounts(1,i,cpg)>=pe:
            return i
    return int(np.ceil(pe))        

In [296]:
def is_dataframe(df):
    if df is not None and type(df)==pd.core.frame.DataFrame:
        return True
    return False

def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==int or type(string)==np.int64 or type(string)==np.float64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),default=np.nan)

def ffloat_list(string_list):
    return list(map(ffloat,string_list))


def get_children(html_content):
    return [item for item in html_content.children if type(item)==bs4.element.Tag]

def get_portfolio(mfid):
    url = "https://www.moneycontrol.com/india/mutualfunds/mfinfo/portfolio_holdings/"+mfid
    page_response = requests.get(url, timeout=10)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    portfolio_table = page_content.find('table', attrs={'class': 'tblporhd'})
    fund_name = page_content.find('h1').text
    return portfolio_table,fund_name

def get_table(portfolio_table):
    portfolio_elems = get_children(portfolio_table)

    table_data = list()
    for row in portfolio_elems:
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","")
            if len(text)==0:
                continue
            elem_descriptor = {'text':text}
            elem_children = get_children(elem)
            if len(elem_children)>0:
                if elem_children[0].has_attr('href'):
                    elem_href = elem_children[0]['href']
                    elem_descriptor['href'] = elem_href

            row_data.append(elem_descriptor)
        table_data.append(row_data)
    return table_data

def get_table_simple(portfolio_table,is_table_tag=True):
    portfolio_elems = portfolio_table.find_all('tr')
    table_data = list()
    for row in portfolio_elems:
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","")
            if len(text)==0:
                continue
            row_data.append(text)
        table_data.append(row_data)
    return table_data

In [5]:
def get_min_and_three_year_from_screener(table):
    min_value = np.inf
    three_year_value = np.inf
    for row in table:
        if len(row)==2:
            if row[0]=='3 Years:':
                three_year_value = ffloat(row[1].replace('%',''))
            cur_value = ffloat(row[1].replace('%',''))
            min_value = min(min_value,cur_value)
    return min_value,three_year_value

def get_quarterly_results(quarterly_results_table):
    qrt = get_table_simple(quarterly_results_table)
    qres = {}
    qres['dates'] = qrt[0]
    qres['sales'] = ffloat_list(qrt[1][1:])
    qres['operating_profit'] = ffloat_list(qrt[3][1:])
    qres['opm_percent'] = ffloat_list(qrt[4][1:])
    qres['interest'] = ffloat_list(qrt[7][1:])
    qres['pbt'] = ffloat_list(qrt[8][1:])
    return qres

def get_annual_results(annual_results):
    qrt = get_table_simple(annual_results)
    qres = {}
    qres['dates'] = qrt[0]
    qres['sales'] = ffloat_list(qrt[1][1:])
    qres['operating_profit'] = ffloat_list(qrt[3][1:])
    qres['opm_percent'] = ffloat_list(qrt[4][1:])
    qres['interest'] = ffloat_list(qrt[6][1:])
    qres['pbt'] = ffloat_list(qrt[8][1:])
    qres['eps'] = ffloat_list(qrt[11][1:])
    return qres

def get_balance_sheet(balance_sheet):
    qrt = get_table_simple(balance_sheet)
    qres = {}
    qres['dates'] = qrt[0]
    qres['borrowings'] = ffloat_list(qrt[3][1:])
    qres['fixed_assets'] = ffloat_list(qrt[6][1:])
    qres['total_assets'] = ffloat_list(qrt[10][1:])
    return qres

def get_cash_flows(cash_flows):
    qrt = get_table_simple(cash_flows)
    qres = {}
    qres['dates'] = qrt[0]
    qres['net_cash_flow'] = ffloat_list(qrt[4][1:])
    return qres

In [165]:


def get_past_prices(sc_id):
    bse_url = "https://www.moneycontrol.com/tech_charts/bse/his/%s.csv"%sc_id
    nse_url = "https://www.moneycontrol.com/tech_charts/nse/his/%s.csv"%sc_id
    
    past_prices_nse = pd.read_csv(nse_url,header=None,names=['open','high','low','close','volume',1,2,3,4])[['open','high','low','close','volume']]
    past_prices_nse.index = pd.to_datetime(past_prices_nse.index)
    
    past_prices_bse = pd.read_csv(bse_url,header=None,names=['open','high','low','close','volume',1,2,3,4])[['open','high','low','close','volume']]
    past_prices_bse.index = pd.to_datetime(past_prices_bse.index)
    
    ly = None
    two_year_ago = None
    three_year_ago = None
    five_year_ago = None
    past_prices = past_prices_bse
    for i in range(12):
        try:
            if ly is None:
                ly_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(364+i, unit='d'))
                ly = past_prices.loc[[ly_t]]
            if two_year_ago is None:
                two_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(730+i, unit='d'))
                two_year_ago = past_prices.loc[[two_year_ago_t]]
            if three_year_ago is None:
                three_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(1095+i, unit='d'))
                three_year_ago = past_prices.loc[[three_year_ago_t]]
            if five_year_ago is None:
                five_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(1825+i, unit='d'))
                five_year_ago = past_prices.loc[[five_year_ago_t]]
        except Exception as e:
            pass
       
    past_prices = past_prices_nse
    for i in range(12):
        try:
            if ly is None:
                ly_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(364+i, unit='d'))
                ly = past_prices.loc[[ly_t]]
            if two_year_ago is None:
                two_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(730+i, unit='d'))
                two_year_ago = past_prices.loc[[two_year_ago_t]]
            if three_year_ago is None:
                three_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(1095+i, unit='d'))
                three_year_ago = past_prices.loc[[three_year_ago_t]]
            if five_year_ago is None:
                five_year_ago_t = pd.to_datetime(past_prices.iloc[-1:].index.values[0] - pd.to_timedelta(1825+i, unit='d'))
                five_year_ago = past_prices.loc[[five_year_ago_t]]
        except Exception as e:
            pass
    
    if len(past_prices_nse)>=len(past_prices_bse):
        past_prices = past_prices_nse
    else:
        past_prices = past_prices_bse
    res = {"all_past_prices":past_prices,"last_year":ly,"two_year_ago":two_year_ago,
            "three_year_ago":three_year_ago,"five_year_ago":five_year_ago}
    return res

### Lib 2

In [232]:

def get_scrip_info(url):
    original_url = url
    key_val_pairs = {}
    key_val_pairs["original_url"] = original_url
    if not url.startswith("http"):
        url = "https://www.moneycontrol.com"+url
    try:
        page_response = requests.get(url, timeout=10)
        page_content = BeautifulSoup(page_response.content, "html.parser")
        scrip_name = None
        name_divs = page_content.find_all('div',attrs={'class':'gry10'})
        for nd in name_divs:
            texts = list(map(lambda x:x.strip(),nd.text.split(" ")))
            if "NSE:" in texts:
                scrip_name = texts[texts.index("NSE:")+1]
                scrip_name = re.sub('[^0-9a-zA-Z&\-]+', '', scrip_name)
        if scrip_name is None or len(scrip_name.strip())==0 or "ETF" in scrip_name:
            key_val_pairs['failure'] = True
            key_val_pairs['err'] = "%s is not named on NSE"%url
            print(key_val_pairs['err'])
            return key_val_pairs
        
        content_div_text = page_content.find('div',attrs={'id':'content_full'}).text
        if "not listed" in content_div_text or "not traded" in content_div_text:
            key_val_pairs['failure'] = True
            key_val_pairs['err'] = "%s is not listed on both BSE and NSE"%url
            print(key_val_pairs['err'])
            return key_val_pairs
        price = ffloat(page_content.find('div',attrs={'id':'Nse_Prc_tick_div'}).text.split(" ")[0].replace(',',''))
        name = page_content.find('h1',attrs={'class':'company_name'}).text
        
        screener_url = "https://www.screener.in/company/%s/"%scrip_name
        screener_page_response = requests.get(screener_url, timeout=10)
        
        if screener_page_response.status_code>299:
            key_val_pairs['failure'] = True
            key_val_pairs['err'] = "No Screener URL: %s"%screener_url
            print(key_val_pairs['err'])
            return key_val_pairs
        screener_page_content = BeautifulSoup(screener_page_response.content, "html.parser")
        screener_name = get_children(get_children(screener_page_content.find('nav',attrs={'id':'fixed-scroll-aid-bar'}))[0])[0].text.strip()

        sector = get_children(screener_page_content.find('h1'))[0].text.replace("/",'').strip()
        yearly_high = page_content.find('span',attrs={'id':'n_52high'}).text.strip()
        yearly_low = page_content.find('span',attrs={'id':'n_52low'}).text.strip()
        html_data_content = page_content.find('div', attrs={'id': 'mktdet_1'})
        petable = get_table(get_children(html_data_content)[0])
        pbtable = get_table(get_children(html_data_content)[1])
        
        
        dma_table = get_table_simple(page_content.find('div',attrs={'id':'acc_hd2'}).find_all('table')[2])
        
        thirty_dma = None
        fifty_dma = None
        one_fifty_dma = None
        two_hundred_dma = None
        if len(dma_table[1])==0:
            thirty_dma = dma_table[1][1]
        if len(dma_table[2])==0:
            fifty_dma = dma_table[2][1]
        if len(dma_table[3])==0:
            one_fifty_dma = dma_table[3][1]
        if len(dma_table[4])==0:
            two_hundred_dma = dma_table[4][1]
        
        
        
        
        
        volume = ffloat(page_content.find('span',attrs={'id':'nse_volume'}).text)
        
        sc_id = page_content.find('input',attrs={'id':'sc_id'}).get('value').lower()
        
        past_prices = get_past_prices(sc_id)
        
        l_yp = None
        two_yp = None
        three_yp = None
        five_yp = None
        gain_loss_l_yp = None
        gain_loss_two_yp = None
        gain_loss_three_yp = None
        gain_loss_five_yp = None
        
        if is_dataframe(past_prices['last_year']):
            l_yp = past_prices['last_year']['close'].values[0]
            gain_loss_l_yp = (price - l_yp)*100/l_yp
        if is_dataframe(past_prices['two_year_ago']):    
            two_yp = past_prices['two_year_ago']['close'].values[0]
            gain_loss_two_yp = (price - two_yp)*100/two_yp
        if is_dataframe(past_prices['three_year_ago']):
            three_yp = past_prices['three_year_ago']['close'].values[0]
            gain_loss_three_yp = (price - three_yp)*100/three_yp
        if is_dataframe(past_prices['five_year_ago']):
            five_yp = past_prices['five_year_ago']['close'].values[0]
            gain_loss_five_yp = (price - five_yp)*100/five_yp
        
        
        
        
        
        quarterly_results = get_quarterly_results(screener_page_content.find('section',attrs={'id':'quarters'}).find('table'))
        
        annual_results_table = screener_page_content.find('section',attrs={'id':'profit-loss'}).find('table',attrs={'class':'data-table'})
        annual_results = None
        if annual_results_table is not None:
            annual_results = get_annual_results(annual_results_table)
        
        csg_table = get_table_simple(screener_page_content.find('section',attrs={'id':'profit-loss'}).find_all('table',attrs={'class':'ranges-table'})[0])
        min_csg,three_year_csg = get_min_and_three_year_from_screener(csg_table)
        cpg_table = get_table_simple(screener_page_content.find('section',attrs={'id':'profit-loss'}).find_all('table',attrs={'class':'ranges-table'})[1])
        min_cpg,three_year_cpg = get_min_and_three_year_from_screener(cpg_table)
        roe_table = get_table_simple(screener_page_content.find('section',attrs={'id':'profit-loss'}).find_all('table',attrs={'class':'ranges-table'})[2])
        min_roe,three_year_roe = get_min_and_three_year_from_screener(roe_table)
        
        balance_sheet = get_balance_sheet(screener_page_content.find('section',attrs={'id':'balance-sheet'}).find('table'))
        cash_flows = get_cash_flows(screener_page_content.find('section',attrs={'id':'cash-flow'}).find('table'))

        data_table = list()
        data_table.extend(petable)
        data_table.extend(pbtable)

        consolidated_html_data_content = page_content.find('div', attrs={'id': 'mktdet_2'})
        consolidated_petable = get_table(get_children(consolidated_html_data_content)[0])
        consolidated_pbtable = get_table(get_children(consolidated_html_data_content)[1])
        consolidated_data_table = list()
        consolidated_data_table.extend(consolidated_petable)
        consolidated_data_table.extend(consolidated_pbtable)

        

        for row in consolidated_data_table:

            k = row[0]['text']
            if len(row)<2:
                v=None
            else:
                v = row[1]['text'].split(" ")[0].replace(',','')
            key_val_pairs[k]=v

        for row in data_table:

            k = row[0]['text']
            if len(row)<2:
                v=None
            else:
                v = row[1]['text'].split(" ")[0].replace(',','')

            if k not in key_val_pairs or not isfloat(key_val_pairs[k]):
                key_val_pairs[k]=v

        key_val_pairs["pe"] = ffloat(key_val_pairs.pop('P/E'))
        key_val_pairs["book_value"] = ffloat(key_val_pairs.pop('BOOK VALUE (Rs)'))
        key_val_pairs["deliverables"] = ffloat(key_val_pairs.pop('DELIVERABLES (%)'))
        key_val_pairs["eps"] = ffloat(key_val_pairs.pop('EPS (TTM)'))
        key_val_pairs["industry_pe"] = ffloat(key_val_pairs.pop('INDUSTRY P/E'))
        if 'MARKET CAP (Rs Cr)' in key_val_pairs:
            key_val_pairs["market_cap"] = key_val_pairs.pop('MARKET CAP (Rs Cr)')
        elif '**MARKET CAP (Rs Cr)' in key_val_pairs:
            key_val_pairs["market_cap"] = key_val_pairs.pop('**MARKET CAP (Rs Cr)')
        key_val_pairs["market_cap"] = ffloat(key_val_pairs["market_cap"])
        key_val_pairs["pb"] = ffloat(key_val_pairs.pop('PRICE/BOOK'))
        key_val_pairs["pc"] = ffloat(key_val_pairs.pop('P/C'))
        key_val_pairs['price'] = ffloat(price)
        key_val_pairs['volume'] = volume
        key_val_pairs["name"] = name
        key_val_pairs["scrip_name"] = scrip_name
        key_val_pairs["yearly_low"] = ffloat(yearly_low)
        key_val_pairs["yearly_high"] = ffloat(yearly_high)

        key_val_pairs["min_csg"] = min_csg
        key_val_pairs["three_year_csg"] = three_year_csg
        key_val_pairs["min_cpg"] = min_cpg
        key_val_pairs["three_year_cpg"] = three_year_cpg
        key_val_pairs["min_roe"] = min_roe
        key_val_pairs["three_year_roe"] = three_year_roe
        key_val_pairs["peg"] = ffloat(key_val_pairs["pe"])/three_year_cpg
        key_val_pairs["min_recovery_year"] = get_year_when_cumulative_profit_over_pe(ffloat(key_val_pairs["pe"]),three_year_cpg)
        key_val_pairs['sector'] = sector
        key_val_pairs['thirty_dma'] = ffloat(thirty_dma)
        key_val_pairs['fifty_dma'] = ffloat(fifty_dma)
        key_val_pairs['one_fifty_dma'] = ffloat(one_fifty_dma)
        key_val_pairs['two_hundred_dma'] = ffloat(two_hundred_dma)
        
        key_val_pairs['l_yp'] = l_yp
        key_val_pairs['two_yp'] = two_yp
        key_val_pairs['three_yp'] = three_yp
        key_val_pairs['five_yp'] = five_yp
        key_val_pairs['gain_loss_l_yp'] = gain_loss_l_yp
        key_val_pairs['gain_loss_two_yp'] = gain_loss_two_yp
        key_val_pairs['gain_loss_three_yp'] = gain_loss_three_yp
        key_val_pairs['gain_loss_five_yp'] = gain_loss_five_yp
        
        key_val_pairs['quarterly_results'] = quarterly_results
        key_val_pairs['annual_results'] = annual_results
        
        key_val_pairs['balance_sheet'] = balance_sheet
        key_val_pairs['cash_flows'] = cash_flows
        key_val_pairs['past_prices'] = past_prices
        key_val_pairs['failure'] = False

        del key_val_pairs['DIV (%)']
        del key_val_pairs['DIV YIELD.(%)']
        del key_val_pairs['FACE VALUE (Rs)']
        del key_val_pairs['Market Lot']
    except Exception as e:
#         print("Final Catch")
        print(original_url)
        #print(screener_url)
#         raise e
        traceback.print_exc()
        key_val_pairs['failure'] = True
        key_val_pairs['err'] = "Error for: %s"%original_url
        print(key_val_pairs['err'])
        return key_val_pairs
        
    return key_val_pairs






In [366]:
def get_all_details(scrip_links_table,percent_col=4,scrip_col=0,threadpool_size=8):
    scrip_details = list()
    percent_col = 4
    scrip_col = 0
    qty_col = 2
    total_value_crores_col = 3
    
    def scrip_detail_collector(row):
        scrip_url = row[scrip_col]['href']
        scrip_detail = get_scrip_info(scrip_url)
        try:
            
            scrip_detail['percent'] = row[percent_col]['text']
            scrip_detail['name'] = row[scrip_col]['text']
            scrip_detail['qty'] = row[qty_col]['text']
            scrip_detail['total_value_crores'] = row[total_value_crores_col]['text']
        except Exception as e:
            print(scrip_url)
        return scrip_detail
        
    pool = ThreadPool(threadpool_size)
    scrip_details = pool.map(scrip_detail_collector, scrip_links_table)
    scrip_details = list(filter(lambda x:x is not None,scrip_details))
    length1 = len(scrip_details)
    scrip_details = list(filter(lambda x:not x['failure'],scrip_details))
    length2 = len(scrip_details)
    print("Scrips which failed to fetch = %s"%(length1-length2))
    scrip_details = pd.DataFrame.from_records(scrip_details)
    numeric_cols = ['book_value', 'price','deliverables', 'eps', 'industry_pe', 
                    'market_cap', 'pb', 'pc', 'pe', 'percent','qty','total_value_crores',
                   'yearly_high','yearly_low','min_csg','three_year_csg','min_cpg','three_year_cpg',
                   'min_roe','three_year_roe','peg','min_recovery_year']
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    return scrip_details

### MF Analysis

In [8]:
# get avg pe and pb and deliverables
# get avg market cap
# current nav
# current aum
# last five year growth rate
# mfid = "MSB532"
# nifty MKM321 MSB1174
# Bluechip SBI MSB532

In [363]:
def comparative_analysis(fund_list,threadpool_size=8):
    fund_details = list()
    for fund in fund_list:
        portfolio_table,fund_name = get_portfolio(mfid = fund)
        table_data = get_table(portfolio_table)
        scrip_details = get_all_details(table_data[1:],threadpool_size=threadpool_size)
        pe = np.dot(scrip_details['price'].fillna(0),scrip_details['percent'])/np.dot(scrip_details['eps'].fillna(0),scrip_details['percent'])
        three_year_cpg = np.dot(scrip_details['three_year_cpg'].fillna(0),scrip_details['percent']/100)
        peg = pe/three_year_cpg
        pb = np.dot(scrip_details['price'].fillna(0),scrip_details['percent'])/np.dot(scrip_details['book_value'].fillna(0),scrip_details['percent'])
        aum = np.sum(scrip_details['total_value_crores'])
        avg_market_cap = np.dot(scrip_details['market_cap'].fillna(0),scrip_details['percent']/100)
        min_recovery_year = get_year_when_cumulative_profit_over_pe(pe,three_year_cpg)
        fund_detail = {"name":fund_name,"pe":pe,"peg":peg,"pb":pb,"aum":aum,"avg_market_cap":avg_market_cap,"three_year_cpg":three_year_cpg,"min_recovery_year":min_recovery_year}
        fund_details.append(fund_detail)
    return pd.DataFrame.from_records(fund_details)

In [367]:
comparative_analysis(['MSB532','MKM321'],threadpool_size=48)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scrips which failed to fetch = 0


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scrips which failed to fetch = 0


,aum,avg_market_cap,min_recovery_year,name,pb,pe,peg,three_year_cpg
0,4452.41,162273.135301,15,SBI Blue Chip Fund - Direct Plan (G),5.718309,34.322055,3.011836,11.395725
1,507.51,299322.601164,14,Kotak Nifty ETF,4.662887,28.591722,2.565105,11.146415


### Company Analysis and Filtering 



#### Criterias - Current
- both NSE and BSE listing
- Not PSU
- Market Cap >= 200 Cr
- Low P/E, pe<20 in all cases, pe wrt to MarketCap and eps increase
- Low debt/Equity Ratio, de<1 in all cases
- P/B ratio
- Current Ratio > 1
- OPM % latest quarter > 10% 
- PEG < 2 , recovery by cumulative profit sum in less than 8 years
- Recovery years on investment by cumulative profits or by pure earnings < 10

#### Criterias - Over time 
- Increasing Sale YoY
- Decreasing d/e ratio YoY
- Increasing eps YoY
- No Steep Price increase in last 2 years

#### Criterias - Buy Now
- Below 30,100 DMA
- Low Volumes 

#### Find Good Companies from MF
- Use above criterias

#### Actual P/E function

In [309]:
def get_pe_filter(params={"mcap":[1e2,1e3,5e3,1e4,2e4],"pe":[1,5,10,15,20],"mcap_lower_limit":1e2,"pe_upper_limit":25}):
    def filter_fn(stock_detail):
        x=params['mcap']
        y=params['pe']
        pe = ffloat(stock_detail['pe'])
        mcap = ffloat(stock_detail['market_cap'])
        
        if np.isnan(pe) or np.isnan(mcap):
            return False
        if pe>params['pe_upper_limit']:
            return False
        if mcap<params['mcap_lower_limit']:
            return False
        
        right = np.searchsorted(params['mcap'],mcap)
        
        
        if right==0:
            right=right+1
        
        if right==len(x):
            right=right-1
        
        left = right-1
        coefficients = np.polyfit([x[left],x[right]], [y[left],y[right]], 1)
        polynomial = np.poly1d(coefficients)
        pe_value = polynomial(mcap)
        if pe<=pe_value:
            return True
        return False
        
    return filter_fn

#### Testing for P/E and marketcap

In [307]:
pe_list = list()
pe_set = set()
pe_comparator = get_pe_filter()
for pe in [1,2,3,4,5,6,7,9,10,11,13]:
    for mcap in np.arange(1e2,2e3,1e2):
        if pe not in pe_set:
            if pe_comparator({"pe":pe,"market_cap":mcap}):
                pe_list.append({"mcap":mcap,"pe":pe})
                pe_set.add(pe)
pd.DataFrame.from_records(pe_list)

,mcap,pe
0,100.0,1
1,400.0,2
2,600.0,3
3,800.0,4
4,1000.0,5
5,1900.0,6


In [308]:
pe_list = list()
pe_set = set()
pe_comparator = get_pe_filter()
for pe in [1,3,7,9,10,11,13,15,17,18,19,20,21,22,25,30]:
    for mcap in np.arange(1e3,2e4,1e3):
        if pe not in pe_set:
            if pe_comparator({"pe":pe,"market_cap":mcap}):
                pe_list.append({"mcap":mcap,"pe":pe})
                pe_set.add(pe)
pd.DataFrame.from_records(pe_list)


,mcap,pe
0,1000.0,1
1,1000.0,3
2,3000.0,7
3,5000.0,9
4,6000.0,10
5,6000.0,11
6,8000.0,13
7,10000.0,15
8,14000.0,17
9,16000.0,18


#### P/B and Book_value>0

In [331]:
def get_pb_filter(params={"mcap":[1e2,5e2,1e3,2e3,6e3],"pb":[1,2,3,4,5],"pb_upper_limit":5}):
    def filter_fn(stock_detail):
        x=params['mcap']
        y=params['pb']
        pb = ffloat(stock_detail['pb'])
        bv = ffloat(stock_detail['book_value'])
        mcap = ffloat(stock_detail['market_cap'])
        
        if np.isnan(pb) or np.isnan(bv) or pb>params['pb_upper_limit'] or bv<0:
            return False
        
        if pb>params['pb_upper_limit']:
            return False
        
        right = np.searchsorted(params['mcap'],mcap)
        
        
        if right==0:
            right=right+1
        
        if right==len(x):
            right=right-1
        
        left = right-1
        coefficients = np.polyfit([x[left],x[right]], [y[left],y[right]], 1)
        polynomial = np.poly1d(coefficients)
        pb_value = polynomial(mcap)
        if pb<=pb_value:
            return True
        return False
        
    return filter_fn

In [332]:
pb_list = list()
pb_comparator_test = get_pb_filter()
pb_set = set()
for pb in [1,1.5,2,2.5,3,4]:
    for mcap in np.arange(1e2,2e3,1e2):
        if pb not in pb_set:
            if pb_comparator_test({'market_cap':mcap,"pb":pb,"book_value":10}):
                pb_list.append({"mcap":mcap,"pb":pb})
                pb_set.add(pb)
pd.DataFrame.from_records(pb_list)

,mcap,pb
0,200.0,1.0
1,400.0,1.5
2,600.0,2.0
3,800.0,2.5
4,1000.0,3.0


In [333]:
pb_comparator_test = get_pb_filter()
pb_list = list()
pb_set = set()
for pb in [1,2,3,4,4.5,5]:
    for mcap in np.arange(1e3,2e4,1e3):
        if pb not in pb_set:
            if pb_comparator_test({'market_cap':mcap,"pb":pb,"book_value":10}):
                pb_list.append({"mcap":mcap,"pb":pb})
                pb_set.add(pb)
pd.DataFrame.from_records(pb_list)

,mcap,pb
0,1000.0,1.0
1,1000.0,2.0
2,1000.0,3.0
3,2000.0,4.0
4,4000.0,4.5
5,6000.0,5.0


#### Getting All Company Links from MoneyControl

In [334]:


def get_stock_urls_from_listing_page(listing_page):
    page_response = requests.get(listing_page, timeout=10)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    urls_table = page_content.find('table',attrs={'class':'pcq_tbl'})
    links = list(map(lambda x:get_children(x)[0]['href'],urls_table.find_all('td')))
    return links

def get_all_links(threadpool_size=8):
    abc = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letters = [letter for letter in abc]
    listing_page_urls = ['https://www.moneycontrol.com/india/stockpricequote/'+letter for letter in letters]
    pool = ThreadPool(threadpool_size)
    all_links = list(more_itertools.flatten(pool.map(get_stock_urls_from_listing_page, listing_page_urls)))
    return all_links

In [245]:
def get_all_company_details(accumulator={},failures=[],size=10000,threadpool_size=8,ignore_failures=True,ignore_success=True):
    # filters is a list of functions returning T/F, They are always
    batch_size = 10*threadpool_size
    all_links = get_all_links()
    if ignore_success:
        all_links = list(set(all_links) - set([scrip['original_url'] for scrip in accumulator.values()]))
    if ignore_failures:
        all_links = list(set(all_links) - set([scrip['original_url'] for scrip in failures]))
    shuffle(all_links)
    all_links = all_links[:size]
    print("Total Links after removing success and failures = %s"%(len(all_links)))
    pool = ThreadPool(threadpool_size)
    batches = int(np.ceil(len(all_links)/batch_size))
    
    for batch_num in range(batches):
        start = batch_num*batch_size
        end = (batch_num+1)*batch_size
        print("start = %s, end = %s"%(start,end))
        this_batch = all_links[start:end]
        scrip_details = pool.map(get_scrip_info, this_batch)
        scrip_details = list(filter(lambda x:x is not None,scrip_details))
        fails = list(filter(lambda x:x['failure'],scrip_details))
        scrip_details = list(filter(lambda x:not x['failure'],scrip_details))
        for scrip in scrip_details:
            accumulator[scrip['scrip_name']] = scrip
        failures.extend(fails)

In [108]:
accumulator={}

In [234]:
failures = []

In [336]:

get_all_company_details(accumulator,failures,size=10000,threadpool_size=48);

Total Links after removing success and failures = 0


In [335]:
len(accumulator.keys())
len(failures)

1581

6795

#### Write to File

In [355]:

import dill
with open('stock_results.pkl', 'wb') as f:
    dill.dump(accumulator, f)
    dill.dump(failures, f)

#### Read from File

In [356]:
with open('stock_results.pkl', 'rb') as f:
    accumulator = dill.load(f)
    failures = dill.load(f)

In [350]:
def filter_companies(all_scrips,filters=[]):
    scrip_details = list(all_scrips.values())
    for filtr in filters:
        scrip_details = list(filter(filtr,scrip_details))
        print(len(scrip_details))
    scrip_details = pd.DataFrame.from_records(scrip_details)
    numeric_cols = ['book_value', 'price','deliverables', 'eps', 'industry_pe', 
                    'market_cap', 'pb', 'pc', 'pe',
                   'yearly_high','yearly_low','min_csg','three_year_csg','min_cpg','three_year_cpg',
                   'min_roe','three_year_roe','peg','min_recovery_year']
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    return scrip_details
    
    

In [351]:
len(list(filter(get_pe_filter(),list(accumulator.values()))))

92

In [352]:
all_company_df = filter_companies(accumulator,filters=[get_pe_filter()])
all_company_df

92


StopIteration: 

StopIteration: 